***Задание №1***

Спроектировать объектную модель сервиса:
* Выделить базовые сущности с которыми будет происходить работа. Пользователь, ML модель, история транзакций/предсказаний, задача для ML модели и т.д.
* При проектирование написать названия классов, названия полей, методов, принимаемых типов данных, модификаторов доступа.
* При проектирование использовать все базовые принципы ООП, а именно инкапсуляцию, наследование для расширения классов и полиморфизм.

In [1]:
import numpy as np
import pandas as pd
import datetime
import random

## ML-модель

In [ ]:
# Простой класс для хранения модели и ее обще описательных характеристик

class MLmodel:
    def __init__(self, model, name: str, version: str, description: str,):
        self.model = model # сама модель
        self.name = name # Название модели
        self.version = version # Версия модели
        self.description = description # Описательная характеристика модели

    def __str__(self):
        print(f"{self.model} \n" \
               f"Название: {self.name} \n"  \
               f"Версия: {self.version} \n" \
               f"Описание: {self.description}")

In [3]:
# Наследуемый класс модели для учета ее параметров использования

class UsableModel(MLmodel):
    def __init__(self, model: MLmodel, name: str, version: str, description: str, price: float):
        super().__init__(model, name, version, description)
        self.price = price # Стоимость использования модели

        self.StatusOperation = None # Статус последнего запуска
        self.Result = None # Результаты работы модели

    def run(self):
        # Запускаем модель и сообщаем о статусе
        self.StatusOperation = bool(random.randint(0, 1)) # Успешность модели пока рандомна - может работать, а может нет
        print('| Модель сработала: ', self.StatusOperation, '|\n' )
        self.Result = 'Результаты работы модели'    

    def __str__(self):
        super().__str__()
        print(f"Стоимость: {self.price} \n"  \
              f"Статус последнего запуска: {self.StatusOperation}") 

In [4]:
CurrModel_1 = UsableModel('', 'Убиватор-3000', '1.01', 'Офигенская модель, прям всем советую только ее',
                          price = 1)

CurrModel_2 = UsableModel('', 'УльтраУбиватор-9000', '3.33', 'Еще более крутая и дорогая модель.',
                          price = 2)

## Биллинг

In [5]:
class PyamentsHistory:

    def __init__(self):
        self.__LastOperationDate = np.NaN # Дата последней удачной операции
        self.__Operations = pd.DataFrame(columns=['Date', 'Operation', 'Val', 'Success']) # Список всех операций и дата свершения

    def get(self):
        return self.__Operations


    def update(self, Date, Operation: str, Val: float, Success: bool):
        LastPos = len(self.get())
        self.__Operations.loc[LastPos, 'Date'] = Date
        self.__Operations.loc[LastPos, 'Operation'] = Operation
        self.__Operations.loc[LastPos, 'Val'] = Val
        self.__Operations.loc[LastPos, 'Success'] = Success

        self.set_lastOperationDate()

    def get_Operations(self):
        return self.__Operations

    def get_lastOperationDate(self):
        return self.__LastOperationDate
    
    def set_lastOperationDate(self, OperationDate = datetime.datetime.now()):
        self.__LastOperationDate = OperationDate
    

In [6]:
class Billing:

    def __init__(self, startbalance: float, FreeLimitPerDay: int):
        self.__balance = startbalance # Стартовый балланс
        self.__FreeLimitToday = FreeLimitPerDay # сколько обесплатных операций осталось на сегодня
        self.__FreeLimitPerDay = FreeLimitPerDay # сколько обесплатных операций осталось на сегодня

        self.__Operations = PyamentsHistory() #история платежных операций



    # Операция пополнения балланса    
    def refund(self, payment: float):
        self.__balance += payment
        print(f"Пополнение баланса на {payment} ед. прошлошло успешно. Текущий балланс — {self.__balance} ед.") 

        self.__Operations.update(Date = datetime.datetime.now(), 
                              Operation = 'Пополнение баланса', 
                              Val = payment,
                              Success = True)          



    # Операция возобновления суточных лимитов
    def refillLimits(self, Num = None):
        # если Num == 0, значит возобновим лимит полностью
        # во всех остальных случаях делаем +Num
        if Num == None:
            self.__FreeLimitToday = self.__FreeLimitPerDay
        else:
            self.__FreeLimitToday += Num

            self.__Operations.update(Date = datetime.datetime.now(), 
                              Operation = 'Восполнение суточного лимита бесплатных операций', 
                              Val = Num,
                              Success = True)   
        
        print(f"Лимиты бесплатных вызовов в день обновлены. Сейчас доступно — {self.__FreeLimitToday}")



    # Операция получения балланса пользователя
    def get_balance(self):
        return self.__balance
    
    # Операция получения балланса пользователя
    def get_limits(self):
        return self.__FreeLimitToday
    
    # Операция получения истории операций билинга
    def get_operations(self):
        return self.__Operations.get_Operations()
    

    # списываем с дневного лимита 
    def set_decLimints(self, limitPayment = 1):
        if self.get_limits() - limitPayment >= 0:
            self.__FreeLimitToday -= limitPayment
            return True
        else:
            return False

    # списываем с дневного лимита 
    def set_decBalance(self, payment = 1):
        if self.get_balance() - payment >= 0:
            self.__balance -= payment
            return True
        else:
            return False

    # Оплата операции
    def pay(self, payment: float):
        ResultOperation = False # все плохо, если ничто этого не изменит

        # Если пользователь не исчерпал бесплатные операции на день - он ничего не платит
        # Если исчерпал - платит
        if self.set_decLimints(payment):
            ResultOperation = True
            Operation = 'Списание в счёт суточного лимита'
        else:    
            if self.set_decBalance(payment):
                ResultOperation = True
                Operation = 'Списание с баланса счета' 

            else:
                Operation = 'Недостаточно срдеств на счете' 
                print(f"Пополните баланс или дождитесь возобновления суточного лимита обращений. \nДля выполнения операции не хватает {abs(self.get_balance() - payment)} ед.")
            
        self.__Operations.update(Date = datetime.datetime.now(), 
                                Operation = Operation, 
                                Val = -payment,
                                Success = ResultOperation)

        return ResultOperation
    

## История операций

In [7]:
class HistoryOperation:
    
    def __init__(self):
        self.LastOperationDate = np.NaN # Дата последней удачной операции
        self.HistoryOperation = pd.DataFrame(columns=['Date', 'Operation', 'Success']) # Список всех операций и дата свершения
        
        self.OperationAll = 0 # Сколько операций сделано за все время
        self.OperationToday = 0 # сколько операций сделано сегодня

        self.Initiation = False # Прошел ли пользователь инициацию первым успешным использованием сервисв


    def get(self):
        return self.HistoryOperation


    def update(self, Date, Operation: str, Success: bool):
        self.set_Initiation()

        LastPos = len(self.get())
        self.HistoryOperation.loc[LastPos, 'Date'] = Date
        self.HistoryOperation.loc[LastPos, 'Operation'] = Operation
        self.HistoryOperation.loc[LastPos, 'Success'] = Success

        self.set_lastOperationDate()

        self.OperationToday += 1
        self.OperationAll += 1


    def get_lastOperationDate(self):
        return self.LastOperationDate
    
    def set_lastOperationDate(self, OperationDate = datetime.datetime.now()):
        self.LastOperationDate = OperationDate


    def get_OperationAll(self):
        return self.OperationAll

    def get_OperationToday(self):
        return self.OperationToday


    def set_OperationToday(self, Num = 0):
        self.OperationToday = Num

    def set_Initiation(self):
        self.Initiation = True
    
    def get_Initiation(self):
        return self.Initiation

## Пользователь

In [8]:
import enum

class UserRole(enum.IntEnum):
    superAdmin = 0
    simpleUser = 1
    megaUser = 2
    gigaUser = 3

In [9]:
class User:

    def __init__(self, name: str, email: str, role, startbalance: int, FreeLimitPerDay: int):
        self.name = name # имя пользователя
        self.email = email # электронный адрес для взаимодействия
        self.RegDate = datetime.datetime.now() # Дата-время регистрации
        self.role = UserRole(role) # Роль, по умолчанию "пользователь"
        
        self.billing = Billing(startbalance, FreeLimitPerDay) # Баланс пользователя в деньгах и свободных лимитах

        self.history = HistoryOperation() # история операций пользователя


    def useModel(self, payment, model: MLmodel):
        #Процедура запуска модели, с проверкой возможности запуска, сохранением параметров и списания баланса
        ResultOperation = False # все плохо, если ничто это го не изменит

        if self.billing.pay(payment):
            # Если платеж прошел или мы можем провести операцию бесплатно в рамках лимита
            model.run()
            self.history.update(Date = datetime.datetime.now(),
                                Operation = 'Запуск модели',
                                Success = model.StatusOperation)

            if not model.StatusOperation:
                # Если запуск модели был неудачный, нужно вернуть оплату 
                # Если операция шла за счет бесплатных, то вернем клиенту это деньгами (надеюсь, нас не обонкротят :О))
                self.billing.refund(payment)
            else:
                ResultOperation = True

        return ResultOperation

In [10]:
# Функция вывода профиля пользователя
def showUserDetails(CurrentUser):
    print('Имя пользователя = ', CurrentUser.name)
    print('Эл. почта = ', CurrentUser.email)
    print('Дата регистрации = ', CurrentUser.RegDate)
    print('Роль = ', CurrentUser.role)
    print('Бесплатных операций в день = ', CurrentUser.billing.get_limits())

    print()
    print('Балланс = ', CurrentUser.billing.get_balance())
    print('Бесплатных операций осталось = ', CurrentUser.billing.get_limits())
    print('Последняя операция = ', CurrentUser.history.get_lastOperationDate())
    print('Всего операций = ', CurrentUser.history.get_OperationAll())
    print('Операций cегодня = ', CurrentUser.history.get_OperationToday())


    print()
    print('Иницализация = ', CurrentUser.history.get_Initiation())



## Демонстрация работы

In [11]:
# Создание пользователя
CurrentUser = User(name = 'dmagog',
                   email = 'georgy-mamarin@mail.ru',
                   role = 1,
                   startbalance = 5,
                   FreeLimitPerDay = 3)

CurrentUser

In [12]:
# Для демонстрации сделаем выбор модели случайным образом
if bool(random.randint(0, 1)): CurrModel = CurrModel_1
else: CurrModel = CurrModel_2

#Пользователь вызывает модель
CurrentUser.useModel(model = CurrModel, 
                     payment = CurrModel.price)


print()
print('Балланс = ', CurrentUser.billing.get_balance())
print('Бесплатных операций осталось = ', CurrentUser.billing.get_limits())
print('Последняя операция = ', CurrentUser.history.get_lastOperationDate())
print('Всего операций = ', CurrentUser.history.get_OperationAll())
print('Операций cегодня = ', CurrentUser.history.get_OperationToday())

print()
print(CurrentUser.history.get())

print()
print(CurrentUser.billing.get_operations())

| Модель сработала:  False |

Пополнение баланса на 2 ед. прошлошло успешно. Текущий балланс — 7 ед.

Балланс =  7
Бесплатных операций осталось =  1
Последняя операция =  2025-03-26 12:44:12.885986
Всего операций =  1
Операций cегодня =  1

                         Date      Operation Success
0  2025-03-26 12:44:12.971803  Запуск модели   False

                         Date                         Operation Val Success
0  2025-03-26 12:44:12.970695  Списание в счёт суточного лимита  -2    True
1  2025-03-26 12:44:12.972078                Пополнение баланса   2    True


In [13]:
# Пополнить баланс
CurrentUser.billing.refund(5)

Пополнение баланса на 5 ед. прошлошло успешно. Текущий балланс — 12 ед.


In [14]:
# Сбросить счетчик бесплатных генераций в день
CurrentUser.billing.refillLimits()

Лимиты бесплатных вызовов в день обновлены. Сейчас доступно — 3


In [15]:
showUserDetails(CurrentUser)

Имя пользователя =  dmagog
Эл. почта =  georgy-mamarin@mail.ru
Дата регистрации =  2025-03-26 12:44:12.952386
Роль =  1
Бесплатных операций в день =  3

Балланс =  12
Бесплатных операций осталось =  3
Последняя операция =  2025-03-26 12:44:12.885986
Всего операций =  1
Операций cегодня =  1

Иницализация =  True
